In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import os
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids 
import pandas as pd
import geopandas as gpd

/home/jace/workspace/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Laos Model Rollout Part 2 (Generate Roll-out Grids) 

This notebook generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

## Set country-specific parameters

In [3]:
REGION = "laos"
ADMIN_LVL = 'ADM2'
ZOOM_LEVEL = 14

## Generate Grids

In [4]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, 
    admin_lvl=ADMIN_LVL, 
    quadkey_lvl=ZOOM_LEVEL, 
    use_cache=True
)

2023-03-10 19:33:19.433 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:264 - No cached grids file found. Generating grids file :/home/jace/.cache/geowrangler/quadkey_grids/laos_14_ADM2_populated_admin_grids.geojson
2023-03-10 19:33:19.435 | DEBUG    | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:281 - Loading boundaries for region laos and admin level ADM2
2023-03-10 19:33:20.564 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:287 - Generating grids for region laos and admin level ADM2 at quadkey level 14


2023-03-10 19:34:20.913 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:293 - Generated 44335 grids for region laos and admin level ADM2 at quadkey level 14
2023-03-10 19:34:20.923 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:299 - Assigning grids to admin areas using metric crs epsg:3857
2023-03-10 19:34:30.095 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:305 - Getting laos population data for filtering grids
2023-03-10 19:34:31.262 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:307 - Computing population zonal stats per grid
2023-03-10 19:34:31.264 | INFO     | povertymapping.rollout_grids:compute_raster_stats:98 - Creating raster zonal stats for 44335 grids for file size 8294976524


## Explore Populated Grids

In [6]:
admin_grids_gdf.info() 

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 22133 entries, 0 to 44334
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geometry    22133 non-null  geometry
 1   quadkey     22133 non-null  object  
 2   shapeName   22133 non-null  object  
 3   shapeISO    22133 non-null  object  
 4   shapeID     22133 non-null  object  
 5   shapeGroup  22133 non-null  object  
 6   shapeType   22133 non-null  object  
 7   pop_count   22133 non-null  float64 
dtypes: float64(1), geometry(1), object(6)
memory usage: 1.5+ MB


In [7]:
admin_grids_gdf.head(2)

,geometry,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count
0,"POLYGON ((100.06348 20.34463, 100.06348 20.365...",13220113003212,Tonpheung,None,LAO-ADM2-3_0_0-B114,LAO,ADM2,1139.814456
1,"POLYGON ((100.08545 20.36523, 100.08545 20.385...",13220113003211,Tonpheung,None,LAO-ADM2-3_0_0-B114,LAO,ADM2,18.897820


In [ ]:
admin_grids_gdf.explore()

## Save to file

In [9]:
rollout_date = '-'.join(os.getcwd().split('/')[-2].split('-')[:3])
grid_save_path = f'./{rollout_date}-la-rollout-grids.geojson'
admin_grids_gdf.to_file(grid_save_path, driver='GeoJSON', index=False)